# POI Data Extraction WorkBench

# **Imports**

In [11]:
!pip install requests
!pip install beautifulsoup4
!pip install wikipedia
!pip install pandas
!pip install numpy
!pip install scrapy
!pip install json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


In [12]:
# Imports for data mining
import wikipedia as wikipedia
import  bs4 as BeautifulSoup
import pandas as pd
import numpy as np
import requests
import json
import wikipedia 
import scrapy
from scrapy import Spider
from scrapy import Request
from scrapy.crawler import CrawlerProcess
from datetime import date

# **Clean Wiki URL Data**

In [13]:

#Create DataFrame for reading in clean API data
title = '2020_deaths_' # CHANGE THIS!
filename = '/content/2020_deaths_wiki_urls_api10_05_2022.csv' # CHANGE THIS!
df = pd.read_csv(filename) # import Extracted API URLs
# Clean dataframe drop null values
df.dropna()
# Remove duplicates
df.drop_duplicates(keep='first')
df.count()
print(df)

      https://www.wikidata.org/w/api.php?action=wbgetentities&sites=enwiki&titles=Category:2020_suicides&props=descriptions&languages=en&format=json
0      https://www.wikidata.org/w/api.php?action=wbge...                                                                                            
1      https://www.wikidata.org/w/api.php?action=wbge...                                                                                            
2      https://www.wikidata.org/w/api.php?action=wbge...                                                                                            
3      https://www.wikidata.org/w/api.php?action=wbge...                                                                                            
4      https://www.wikidata.org/w/api.php?action=wbge...                                                                                            
...                                                  ...                                                  

### **Drop Empty Rows and Remove Duplicates**

In [14]:
# Create new data frame and add column header
dfx = df
dfx.columns =['Header']
# Remove rows of that are not persons based on keywords
dfx = dfx[~dfx.Header.str.contains('Category')]
dfx = dfx[~dfx.Header.str.contains('List_of')]
dfx = dfx[~dfx.Header.str.contains('Deaths_in')]
dfx = dfx[~dfx.Header.str.contains('Missing_')]
dfx = dfx[~dfx.Header.str.contains('Unreported_')]
dfx = dfx[~dfx.Header.str.contains('National_')]
dfx_clean = dfx
# Verfiy removed data
dfx_clean.head(5)

,Header
16,https://www.wikidata.org/w/api.php?action=wbge...
17,https://www.wikidata.org/w/api.php?action=wbge...
18,https://www.wikidata.org/w/api.php?action=wbge...
19,https://www.wikidata.org/w/api.php?action=wbge...
20,https://www.wikidata.org/w/api.php?action=wbge...


# **Get Wiki Data API Url**

In [15]:
# Export Clean dataframe to csv for data extraction
dfx_clean.to_csv('CLEAN_' + title + '.csv', header=None, index=False)
print(df)

                                                  Header
0      https://www.wikidata.org/w/api.php?action=wbge...
1      https://www.wikidata.org/w/api.php?action=wbge...
2      https://www.wikidata.org/w/api.php?action=wbge...
3      https://www.wikidata.org/w/api.php?action=wbge...
4      https://www.wikidata.org/w/api.php?action=wbge...
...                                                  ...
12614  https://www.wikidata.org/w/api.php?action=wbge...
12615  https://www.wikidata.org/w/api.php?action=wbge...
12616  https://www.wikidata.org/w/api.php?action=wbge...
12617  https://www.wikidata.org/w/api.php?action=wbge...
12618  https://www.wikidata.org/w/api.php?action=wbge...

[12619 rows x 1 columns]




---



## **Extract Q-values From Wiki API**

In [16]:
# call Wiki JSON API to extract Q-value UIDs
WikiData_List = []
print('Extracing Q-values API urls...')
count = 0
for link in dfx_clean.iterrows():
    UID = requests.get(link[1][0]).json()['entities'].values()
    try: 
        count += 1
        Q_value = list(UID)[0]['id']
        wikiDataURL ='https://www.wikidata.org/wiki/'
        WikiDataAPI  = wikiDataURL + Q_value
        WikiData_List.append(WikiDataAPI)
        print('Total Q-values extracted =', count)
    except:
        pass
        count += 1
        print(count,' API urls had no Q_value to extract' ) 
    finally:
        pass
print('\n Q-values Extracted to WikiData_List \n')
print('Total URLs without Q_values =', count)

Streaming output truncated to the last 5000 lines.
Total Q-values extracted = 7480
Total Q-values extracted = 7481
Total Q-values extracted = 7482
Total Q-values extracted = 7483
Total Q-values extracted = 7484
Total Q-values extracted = 7485
Total Q-values extracted = 7486
Total Q-values extracted = 7487
7489  API urls had no Q_value to extract
Total Q-values extracted = 7490
Total Q-values extracted = 7491
Total Q-values extracted = 7492
Total Q-values extracted = 7493
Total Q-values extracted = 7494
Total Q-values extracted = 7495
Total Q-values extracted = 7496
Total Q-values extracted = 7497
Total Q-values extracted = 7498
Total Q-values extracted = 7499
Total Q-values extracted = 7500
Total Q-values extracted = 7501
Total Q-values extracted = 7502
Total Q-values extracted = 7503
Total Q-values extracted = 7504
Total Q-values extracted = 7505
Total Q-values extracted = 7506
Total Q-values extracted = 7507
Total Q-values extracted = 7508
Total Q-values extracted = 7509
Total Q-valu

### **Add Q-values to dataframe**

In [17]:
# Add Q_values url data to dataframe
df_data = pd.DataFrame(WikiData_List)

In [18]:
# Input file name
print("Enter Project Name:")
# x = input()
x = title
date = date.today()
tm = date.strftime("%m_%d_%y")
data_file = 'Q_value_url_' + x + '_' + tm + '.csv'
print(data_file)

Enter Project Name:
Q_value_url_2020_deaths__10_08_22.csv


## **Q code API's to CSV file**

In [19]:
# Export dataframe to csv for data extraction
print(df_data.head(25))
df_data.to_csv(data_file,  index=False)

                                           0
0   https://www.wikidata.org/wiki/Q102047119
1   https://www.wikidata.org/wiki/Q103820037
2     https://www.wikidata.org/wiki/Q4569959
3    https://www.wikidata.org/wiki/Q90564126
4     https://www.wikidata.org/wiki/Q2655701
5    https://www.wikidata.org/wiki/Q85835789
6     https://www.wikidata.org/wiki/Q2847124
7     https://www.wikidata.org/wiki/Q1096378
8     https://www.wikidata.org/wiki/Q1740364
9    https://www.wikidata.org/wiki/Q28478411
10    https://www.wikidata.org/wiki/Q6681281
11    https://www.wikidata.org/wiki/Q1641210
12   https://www.wikidata.org/wiki/Q56798315
13   https://www.wikidata.org/wiki/Q16366590
14   https://www.wikidata.org/wiki/Q17126490
15   https://www.wikidata.org/wiki/Q21970492
16    https://www.wikidata.org/wiki/Q3178720
17    https://www.wikidata.org/wiki/Q7172453
18   https://www.wikidata.org/wiki/Q12222794
19   https://www.wikidata.org/wiki/Q92606256
20     https://www.wikidata.org/wiki/Q310203
21   https